# 적합도 검정

## Run test
관측치가 무작위로 추출되었는지를 판단
- 귀무가설: 무작위로 추출되었다.
- 대립가설: 무작위가 아니다. (p-value < 0.05)

In [2]:
from statsmodels.sandbox.stats.runs import Runs
import numpy as np

In [3]:
x = [1,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,0,0,0]
x = np.array(x)

statistic, pvalue = Runs(x).runs_test()
print(statistic)
print(pvalue)

-1.8277470669267506
0.06758752074917526


결과해석
- 유의수준 0.05라면, p-value > 0.05 이므로 귀무가설 채택. 즉, 무작위로 추출되었다.
- 유의수준 0.1이라면, p-value < 0.1 이므로 귀무가설 기각. 즉, 무작위로 추출되지 않았다.

## Kolmogorov-Smirnov test
관측치가 가정한 분포와 적합한지 검정.

ex) 가정한 분포가 정규분포라면
- 귀무가설: 관측치의 분포는 정규분포이다.
- 대립가설: 관측치의 분포는 정규분포가 아니다.

In [4]:
from statsmodels.stats.diagnostic import kstest_normal
import numpy as np 

x = [88, 75, 79, 84, 68, 51, 70, 75, 88, 90, 92, 88, 63, 72, 94, 80, 78, 98, 81, 67, 85, 87, 79, 81, 85, 48, 79, 86, 53, 100, 
     87, 80, 80, 32, 60, 75, 62, 82, 40, 57]
x = np.array(x)

# dist: norm or exp
kstest_normal(x, dist='norm')

/home/jmhwang/miniconda3/envs/adp/lib/python3.7/site-packages/statsmodels/stats/_lilliefors.py:272: FutureWarning: The default pvalmethod will change from "approx" to "table" after 0.11. The "table" method uses values from a very large simulation and is more accurate.  Explicitly set this parameter to "approx" or "table" to silence this warning
  warnings.warn(msg, FutureWarning)


(0.16508249090030575, 0.007697422359967707)

결과해석

- 유의수준이 0.05일때, p-value < 0.05이므로 귀무가설 기각. 즉, 관측치는 정규분포를 따르지 않는다.

# 동질성 검정

## Wilcoxon signed ranks test
paired sample t-test에서 n < 30일 경우에 사용

ex) 기존 물류 알고리즘 보다 개선되었다고 알려진 새로운 물류 경로 알고리즘을 도입해 배송시간을 단축하고자 한다.
    - 귀무가설: 기존과 신규의 배송시간 평균 차이는 없다.
    - 대립가설: 기존과 신규의 배송시간 평균 차이가 있다.

In [18]:
from scipy.stats import wilcoxon

In [19]:
x = [10, 30, 9, 21, 35, 12, 17, 14, 15, 16]
y = [8, 27, 16, 25, 30, 13, 11, 21, 13, 34]

wilcoxon(x, y)

WilcoxonResult(statistic=22.0, pvalue=0.5745648243125437)

특정 평균을 검정하고 싶으면 아래처럼 사용

In [23]:
# wilcoxon(data - mu, alternative="...")

결과해석
- 유의수준 0.05일때, p-value > 0.05 이므로 귀무가설 채택. 즉, 새로운 알고리즘의 배송시간과 차이가 없다.

## Mann-Whitney U test
- 2개의 독립된 집단간의 특정 값의 평균 비교
- 2개의 독립된 집단이 정규분포를 따르지 않을 때 사용한다.

ex) 반도체 회사V에서는 공장A와 공장B에서 반도체를 생산하고 있다. 이 때, 2개의 공장의 생산 효율에 차이가 있는지를 확인하고 싶다.
    - 귀무가설: 두 공장의 생산효율의 차이가 없다.
    - 대립가설: 두 공장의 생산효율의 차이가 있다. (다르다. 그러나 어느 공장이 더 낫다는 검정으로 알 수 없음)

In [7]:
from scipy.stats import mannwhitneyu
import pandas as pd

In [8]:
x = [12, 11, 13, 14, 15]
y = [16, 15, 17, 19, 20]

In [9]:
mannwhitneyu(x, y)

MannwhitneyuResult(statistic=0.5, pvalue=0.007985348176890053)

결과해석

- 유의수준 0.05일때, pvalue < 0.05 이므로 귀무가설 기각. 즉, 차이가 있다.

In [10]:
xy = pd.DataFrame(x+y, columns=['생산량'])
xy['공장이름'] = ["A", "A", "A", "A", "A", "B", "B", "B", "B", "B"]
xy['생산량 순위'] = xy['생산량'].rank(ascending=False)
print(xy.groupby('공장이름').mean())

       생산량  생산량 순위
공장이름              
A     13.0     7.9
B     17.4     3.1


결과해석
- 실제 순위를 매겨보니 B가 낫다.

## Kruskal-Wallis H test
- 3개 이상의 독립 표본 집단간 분포가 동일한지 검정
    - 귀무가설: 집단 간 분포는 동일하다.
    - 대립가설: 집단 간 분포는 동일하지 않다.

In [11]:
import pandas as pd
from scipy.stats import kruskal

a = [35, 41, 45, 42, 33, 36, 47, 45, 31, 32, 40, 44]
b = [40, 38, 44, 48, 45, 46, 42, 39, 40, 41, 38, 47]
c = [30, 34, 38, 39, 40, 41, 38, 37, 40, 41, 39, 38]

In [12]:
kruskal(a, b, c)

KruskalResult(statistic=6.047476974964328, pvalue=0.04861911622342764)

결과해석
- 유의수준 0.05일 때, p-value < 0.05이므로 귀무가설을 기각할 수 있다. 따라서 3개의 집단의 분포는 동일하지 않다.

## Friedman test

- 실험유형과 집단유형으로 구분된 여러 집단들의 서열정보를 사용하여 실험효과를 검증하는 방법

ex)

- A쇼핑 본사에서는 멤버십 VIP 고객들을 대상으로 새로운 혜택을 제공하고자 한다.<br>
샘플증정, 포인트 추가, 무료배송, 할인쿠폰 등 4가지 혜택에 대한 5개 지역별 고객들에 대한 사전 선호도 조사를 실시하였다.
    - 귀무가설: 혜택별 고객 선호도 평가에 차이는 없다.
    - 대립가설: 혜텍별 고객 선호도 평가에 차이는 있다.

In [13]:
from scipy.stats import friedmanchisquare

a = [1,2,1,1,2]
b = [3,3,3,2,1]
c = [2,4,4,4,3]
d = [4,1,2,3,4]

friedmanchisquare(a, b, c, d)

FriedmanchisquareResult(statistic=6.359999999999999, pvalue=0.09535032301698126)

결과해석
- 유의수준 0.05일때, p-value > 0.05 이므로 귀무가설 채택. 즉 혜택별 고객 선호도 평가에 차이가 없다.

# 상관성 검정

### 스피어만 서열상관분석

In [16]:
# import scipy.stats
# scipy.stats.spearmanr(x, y).correlation

### 피어슨 서열상관분석

In [17]:
# import scipy.stats
# scipy.stats.pearsonr(x, y)

### 정준상관분석 (x)

### Kendall rank correlation coefficient

- 귀무가설: H홈쇼핑의 의류브랜드 가격 서열과 판매량 서열은 연관성이 없다.
- 연구가설: H홈쇼핑의 의류브랜드 가격 서열과 판매량 서열은 연관성이 있다.

In [15]:
from scipy.stats import kendalltau

x = [5, 2, 4, 1, 3]
y = [4, 1, 3, 2, 5]

kendalltau(x, y)

KendalltauResult(correlation=0.39999999999999997, pvalue=0.48333333333333334)

결과해석
- 유의수준 0.05일떄, p-value > 0.05 이므로 귀무가설을 채택할 수 있다. 따라서 서로 연관성이 없다고 해석할 수 있다.
- 즉, 판매량 저조의 문제점을 다른 곳에서 찾아야한다.